In [140]:
import pandas as pd
import os
import glob
import plotly
import dash
from posicionamento import posicionamento_partidos

In [141]:
import plotly.express as px
dfsss = px.data.stocks()
fig = px.line(dfsss, x='date', y="GOOG")
fig.show()

In [120]:
all_files = glob.glob(os.path.join("dados/*.csv"))
all_files
li = []

for file in all_files:
    df = pd.read_csv(file, index_col=None, header=0, sep=";")
    li.append(df)

df = pd.concat(li, axis=0, ignore_index=True)
df = df.drop(["uriVotacao", "deputado_uri", "deputado_uriPartido", "deputado_urlFoto", "deputado_idLegislatura"], axis=1)

df_nan = df[df["voto"].isnull()].groupby("idVotacao").count()
df = df[~df["idVotacao"].isin(df_nan.index)]

df["voto"].replace({"Sim": 1, "Não": -1, "Abstenção": 0, 'Artigo 17':0, 'Obstrução':0, 'Favorável com restrições': 0, 'Branco': 0 }, inplace=True)
df["voto_count"] = 1

In [121]:
df_nan = df[df["voto"].isnull()].groupby("idVotacao").count()
ids = df_nan.index
ids

Index([], dtype='object', name='idVotacao')

In [122]:
df[df["idVotacao"].isin(ids)]

,idVotacao,dataHoraVoto,voto,deputado_id,deputado_nome,deputado_siglaPartido,deputado_siglaUf,voto_count


In [123]:
for id in ids:
    print(df[df["idVotacao"] == id]["voto"].unique())

In [125]:
n_df = df.copy()

In [126]:
n_df["posicionamento"] = n_df["deputado_siglaPartido"]
n_df["posicionamento"].replace(posicionamento_partidos, inplace=True)
n_df

,idVotacao,dataHoraVoto,voto,deputado_id,deputado_nome,deputado_siglaPartido,deputado_siglaUf,voto_count,posicionamento
0,37897-3,2001-12-11T19:49:15,1,73886,Adão Pretto,PT,RS,1,Esquerda
1,37897-3,2001-12-11T19:29:35,1,74328,Adauto Pereira,PFL,PB,1,Centro-direita
2,37897-3,2001-12-11T19:26:34,1,74450,Adolfo Marinho,PSDB,CE,1,Centro
3,37897-3,2001-12-11T19:25:14,0,74646,Aécio Neves,PSDB,MG,1,Centro
4,37897-3,2001-12-11T19:26:59,1,73765,Affonso Camargo,PFL,PR,1,Centro-direita
...,...,...,...,...,...,...,...,...,...
1329477,945554-186,2022-04-07T13:21:12,1,178889,Zé Carlos,PT,MA,1,Esquerda
1329478,945554-186,2022-04-07T13:22:05,1,204559,Zé Neto,PT,BA,1,Esquerda
1329479,945554-186,2022-04-07T13:26:44,-1,160632,Zé Silva,SOLIDARIEDADE,MG,1,Centro
1329480,945554-186,2022-04-07T13:21:07,-1,204517,Zé Vitor,PL,MG,1,Centro-direita


In [127]:
n_df["voto"].unique()

array([ 1,  0, -1], dtype=int64)

In [167]:
pos_votos = n_df.groupby(["posicionamento", "voto"], as_index=False).count().drop(["idVotacao", "dataHoraVoto", "deputado_id", "deputado_nome", "deputado_siglaPartido", "deputado_siglaUf"], axis=1)

In [168]:
pos_votos

,posicionamento,voto,voto_count
0,Centro,-1,177283
1,Centro,0,11807
2,Centro,1,217254
3,Centro-direita,-1,98261
4,Centro-direita,0,9115
5,Centro-direita,1,115931
6,Centro-esquerda,-1,69139
7,Centro-esquerda,0,7908
8,Centro-esquerda,1,84998
9,Direita,-1,146730


In [173]:
pos_votos["voto"].replace({-1: "Não", 0: "Abstenção", 1: "Sim"}, inplace=True)

In [174]:
# pos_votos.query('posicionamento == "Centro" and voto == 1')
pos_votos

,posicionamento,voto,voto_count
0,Centro,Não,177283
1,Centro,Abstenção,11807
2,Centro,Sim,217254
3,Centro-direita,Não,98261
4,Centro-direita,Abstenção,9115
5,Centro-direita,Sim,115931
6,Centro-esquerda,Não,69139
7,Centro-esquerda,Abstenção,7908
8,Centro-esquerda,Sim,84998
9,Direita,Não,146730


In [187]:
import plotly.express as px
fig = px.bar(
    pos_votos, 
    x="posicionamento",
    color="voto",
    color_discrete_map={
        "Sim": 'rgb(0, 122, 255)',
        "Não": 'rgb(220, 20, 60)',
        "Abstenção": 'rgb(236, 164, 54)'
    },
    y="voto_count"
)
fig.show()